In [ ]:
### XGBOOST with non-parametric confidence intervals

import pandas as pd
import numpy as np
import snowflake.connector
import xgboost as xgb
import pickle
import matplotlib.pyplot as plt

In [ ]:
#Connect to Snowflake
ctx = snowflake.connector.connect("**REMOVED**")

In [ ]:
# Download data

sql="""SELECT DIED, GENDER, AGE, UMC_30, UMC_HEART, UMC_DEMENTIA, UMC_CKD, UMC_INHERITED_METABOLIC, UMC_HYP, UMC_DM2, UMC_OBESITY, UMC_CBD, 
UMC_COPD, UMC_HYL, UMC_ASTHMA, UMC_CANCER, UMC_DM1, UMC_LIVER, UMC_PREGNANCY, UMC_PULM_FIB, UMC_RHEUMATOID_ARTHRITIS, UMC_PARKINSONS, UMC_PANCREATITIS, UMC_DEV_BEH_DISORDER,
UMC_PROSTATE_CANCER, UMC_LUNG_CANCER, UMC_COLORECTAL_CANCER, UMC_BREAST_CANCER, UMC_IMMUNE_DEF, UMC_LYMPHOMA_MYELOMA, UMC_LUPUS, UMC_MULTIPLE_SCLEROSIS,
UMC_IMMUNE_SUPPRESSANTS, UMC_TRANS, NEWCASE_DATE, DAYS_HISTORY_BEFORE_COVID, DAYS_SINCE_2020, YOB, DIED_IN_2020
FROM AIQ_MORTALITY_OFFICEALLY.AIQ_MORTALITY_OFFICEALLY_00051.Ji_patient_all WHERE 
COVID = 1 AND AGE IS NOT NULL"""

df = pd.read_sql(sql,ctx)
#Closes the Connection
ctx.close()

In [ ]:
# Select by age and time horizon:
mdf = df[:]

# Remove censoring-point ages (some providers make everyone above this age exactly this age, also 
# removing (likely delivery-related) spike at 0 and 1 in this data)
mdf = mdf[(mdf.AGE >= 0) & (mdf.AGE != 89) & (mdf.AGE != 80)]

# Check that year of birth and age match roughly:
mdf = mdf[abs((2020-mdf.YOB) - mdf.AGE) <= 1]

# Make days since 2020 into weeks (to avoid weekly cycles)
mdf['WEEKS_SINCE_2020'] = round(mdf['DAYS_SINCE_2020']/7)

# Make sure we use died in 2020:
mdf.describe()
mdf['DIED_IN_2020_FIXED'] = mdf['DIED_IN_2020']

# This fixes an error in which "not died" was coded as missing (see line below for comfirmation that this was happening)
mdf['DIED_IN_2020_FIXED'] = mdf['DIED_IN_2020_FIXED'].fillna(0)
#(mdf.DIED - mdf.DIED_IN_2020_FIXED).describe()

# Save deaths in 2020 or 2021 column
mdf['DIED_IN_2021_OR_2020'] = mdf['DIED']

# Set died in 2020 as default
mdf['DIED'] = mdf['DIED_IN_2020_FIXED']

# Small fix: Drop men with pregnancy-related claims (just a bit over 100 total):
mdf = mdf.drop(mdf[(mdf.GENDER == 2) & (mdf.UMC_PREGNANCY == 1)].index)

# Drop obviously miscoded pregnancy-related claims (based on age):
mdf = mdf.drop(mdf[(mdf.AGE < 10) & (mdf.UMC_PREGNANCY == 1)].index)

# Cut the sample off in mid-December 2020 (doing this here matters for CFR calculations):
mdf = mdf[(mdf.DAYS_SINCE_2020 <= 366-15)]

mdf.describe()

In [ ]:
# Create augmented dataset ("debiased")
mdx = mdf[:]
mdx.describe()

In [ ]:
# Add age group column to mdx
age_bucket_labels = ['0-4 Years', '5-14 Years', '15-24 Years', '25-34 Years', '35-44 Years',
                     '45-54 Years', '55-64 Years', '65-74 Years', '75-84 Years',
                     '85 Years and Over']
age_bucket_bins = [-1, 5, 15, 25, 35, 45, 55, 65, 75, 85, 150]

mdx['AGE_GROUP'] = pd.cut(mdx.loc[:,'AGE'], age_bucket_bins, labels=age_bucket_labels)
mdx_neg_patients['AGE_GROUP'] = pd.cut(mdx_neg_patients.loc[:,'AGE'], age_bucket_bins, labels = age_bucket_labels)

# To inspect;
# mdx[['AGE_GROUP', 'AGE']].head(10)


In [ ]:
# Group mdx df:
cfr_mdx = mdx[['AGE_GROUP', 'DIED', 'GENDER', 'WEEKS_SINCE_2020']].groupby(by=['AGE_GROUP',
                                                                              'GENDER',
                                                                              'WEEKS_SINCE_2020'],
                                                                          as_index=False).agg(['mean', 'count'])

cfr_mdx = cfr_mdx.reset_index(drop=False)
cfr_mdx.columns = [' '.join(col).strip() for col in cfr_mdx.columns.values]
cfr_mdx.rename(columns={'DIED mean': 'CFR',
                       'DIED count': 'CASES'},
              inplace=True)

In [ ]:
# Sort dataset by date within age and gender, and get 5-week average cfr for covid-positive patient data:

# Sort data
cfr_mdx = cfr_mdx.sort_values(by=['AGE_GROUP', 'GENDER', 'WEEKS_SINCE_2020'])

# Break up the df by age group and gender, and get rolling centered average CFR over 5-week periods
cfr_mdx_dfs = []
for age_bucket in age_bucket_labels:
    for gender in [1, 2]:
        cfr_mdx_df = cfr_mdx[(cfr_mdx.AGE_GROUP == age_bucket) & 
                             (cfr_mdx.GENDER == gender)]
        cfr_mdx_df.insert(5, 'CFR_ROLLING', cfr_mdx_df['CFR'].rolling(5, min_periods=1).mean())
       # cfr_mdx_df['CFR_ROLLING'] = cfr_mdx_df['CFR'].rolling(5, min_periods=1).mean() # using above to avoid error
        cfr_mdx_dfs.append(cfr_mdx_df)

# Concatenate all dataframes created above:
cfr_mdx_rolling = pd.concat(cfr_mdx_dfs).reset_index(drop=True)

In [ ]:
# Optional chunk to inspect smoothing and underlying data:
for age_group in age_bucket_labels:
    temp_df = cfr_mdx_rolling[(cfr_mdx_rolling.WEEKS_SINCE_2020 >= 20) & 
                              (cfr_mdx_rolling.GENDER == 1) & 
                              (cfr_mdx_rolling.AGE_GROUP == age_group)]
   
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CFR, color = "darkgray")
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CFR_ROLLING, 
             color = "darkred")
    
    temp_df = cfr_mdx_rolling[(cfr_mdx_rolling.WEEKS_SINCE_2020 >= 20) & 
                              (cfr_mdx_rolling.GENDER == 2) & 
                              (cfr_mdx_rolling.AGE_GROUP == age_group)]
   
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CFR, color = "gray")
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CFR_ROLLING, 
             color = "red")

    plt.title(age_group)
    plt.show()

In [ ]:
# Inspect this dataframe
cfr_mdx.describe()

In [ ]:
# We then read in the national CFR
cfr_usa = pd.read_csv('case_fatality_rate_weekly.csv')

# This was estimated by week, age group and gender based on the CDC patient-line data of diagnosed, scaled to 
# reflect total case counts by week (i.e. assuming the age distribution for the missing cases are similar to 
# those that are not missing) and CDC provisional death counts to covid-19 per week, assuming a 14 day median 
# delay between diagnosis and death. While the patient line data has some categorized as dead, it does not record
# an outcome in most cases, and therefore could not be used directly.

# Make naming consistent
cfr_usa['WEEK'] = cfr_usa['DEATH_WEEK'] # (the week is the week of the recorded cases)
cfr_usa['CFR'] = cfr_usa['DEATHS_TWO_WEEKS_LATER'] / cfr_usa['CASES'] # (we assume median 2 weeks between diagnosis and death)

## Fix gender to conform to big dataset:
cfr_usa['GENDER'] = [1.0 if sex == 'Female' else 2.0 for sex in cfr_usa.SEX]

## Rename columns to conform
cfr_usa.rename(columns={'WEEK': 'WEEKS_SINCE_2020'},
              inplace=True)
cfr_usa.drop(columns='SEX', inplace=True)

In [ ]:
# Sort data
cfr_usa = cfr_usa.sort_values(by=['AGE_GROUP', 'GENDER', 'WEEKS_SINCE_2020'])

# Break up the df by age group and gender, and get rolling centered average CFR over 5-week periods
cfr_usa_dfs = []
for age_bucket in age_bucket_labels:
    for gender in [1, 2]:
        cfr_usa_df = cfr_usa[(cfr_usa.AGE_GROUP == age_bucket) & 
                             (cfr_usa.GENDER == gender)]
        cfr_usa_df.insert(5, 'CFR_ROLLING', cfr_usa_df['CFR'].rolling(5, min_periods=1).mean())
        cfr_usa_dfs.append(cfr_usa_df)

# Concatenate all dataframes created above:
cfr_usa_rolling = pd.concat(cfr_usa_dfs).reset_index(drop=True)
cfr_usa_rolling.describe()

In [ ]:
# We then merge the national CFR and patient-level CFR
cfr_merged = cfr_mdx_rolling.merge(cfr_usa_rolling,
                                  on=['AGE_GROUP',
                                     'GENDER',
                                     'WEEKS_SINCE_2020'],
                                  how='left',
                                  suffixes=['_mdx', '_usa'])
#cfr_merged['N_SAMPLES_NEEDED']
cfr_merged['APPROX_DEATHS'] = cfr_merged['CFR_ROLLING_mdx']*cfr_merged['CASES_mdx']
cfr_merged['N_SAMPLES_NEEDED'] = (cfr_merged['CASES_mdx'] * 
                                  (cfr_merged['CFR_ROLLING_mdx']-cfr_merged['CFR_ROLLING_usa']))/cfr_merged['CFR_ROLLING_usa']

# Check that this works:
cfr_merged['FIXED_CFR'] = (cfr_merged['CASES_mdx'] * cfr_merged['CFR_ROLLING_mdx'])/ (cfr_merged['CASES_mdx'] + cfr_merged['N_SAMPLES_NEEDED'])

print((cfr_merged.FIXED_CFR - cfr_merged.CFR_ROLLING_usa).max())
# Yes!

cfr_merged.describe()

# Move negative N-samples needed to separate column
cfr_merged['N_SAMPLES_EXTRA'] = - cfr_merged['N_SAMPLES_NEEDED']
cfr_merged.loc[(cfr_merged['N_SAMPLES_NEEDED'] < 0), 'N_SAMPLES_NEEDED'] = 0
cfr_merged.loc[(cfr_merged['N_SAMPLES_EXTRA'] < 0), 'N_SAMPLES_EXTRA'] = 0

cfr_merged.describe()

In [ ]:
#cfr_merged = pd.read_csv('DEBIASING_TARGETS.csv')

In [ ]:
# Pretty much all of these are in the 85 years and older category.
#cfr_merged.N_SAMPLES_EXTRA[(cfr_merged['AGE_GROUP'] == '85 Years and Over')].sum()/
#cfr_merged[(cfr_merged.WEEKS_SINCE_2020 == 48)]
#cfr_merged.N_SAMPLES_EXTRA.sum()

In [ ]:
# Optional chunk to inspect additions by age group and gender):
for age_group in age_bucket_labels:
    temp_df = cfr_merged[(cfr_merged.WEEKS_SINCE_2020 >= 20) &
                         (cfr_merged.GENDER == 1) & 
                         (cfr_merged.AGE_GROUP == age_group)]
   
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CASES_mdx, color = "darkgray")
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CASES_mdx + temp_df.N_SAMPLES_NEEDED - temp_df.N_SAMPLES_EXTRA, 
             color = "pink")
    
    temp_df = cfr_merged[(cfr_merged.WEEKS_SINCE_2020 >= 20) &
                         (cfr_merged.GENDER == 2) & 
                         (cfr_merged.AGE_GROUP == age_group)]
   
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CASES_mdx, color = "darkblue")
    plt.plot(temp_df.WEEKS_SINCE_2020, temp_df.CASES_mdx + temp_df.N_SAMPLES_NEEDED - temp_df.N_SAMPLES_EXTRA, 
             color = "blue")

    plt.title(age_group)
    plt.show()

In [ ]:
# Design sampling procedure:
# First: draw subsample from big data to use as base

#Connect to Snowflake
ctx = snowflake.connector.connect(**REMOVED**)

# Download data (this draws a random sample of about 10m)
sql="""SELECT DIED, GENDER, AGE, UMC_30, UMC_HEART, UMC_DEMENTIA, UMC_CKD, UMC_INHERITED_METABOLIC, UMC_HYP, UMC_DM2, UMC_OBESITY, UMC_CBD, 
UMC_COPD, UMC_HYL, UMC_ASTHMA, UMC_CANCER, UMC_DM1, UMC_LIVER, UMC_PREGNANCY, UMC_PULM_FIB, UMC_RHEUMATOID_ARTHRITIS, UMC_PARKINSONS, UMC_PANCREATITIS, UMC_DEV_BEH_DISORDER,
UMC_PROSTATE_CANCER, UMC_LUNG_CANCER, UMC_COLORECTAL_CANCER, UMC_BREAST_CANCER, UMC_IMMUNE_DEF, UMC_LYMPHOMA_MYELOMA, UMC_LUPUS, UMC_MULTIPLE_SCLEROSIS,
UMC_IMMUNE_SUPPRESSANTS, UMC_TRANS, NEWCASE_DATE, DAYS_HISTORY_BEFORE_COVID, DAYS_SINCE_2020, YOB, DIED_IN_2020, COVID
FROM AIQ_MORTALITY_OFFICEALLY.AIQ_MORTALITY_OFFICEALLY_00051.Ji_patient_all SAMPLE (10.0)"""

big_data_df = pd.read_sql(sql,ctx)
#Closes the Connection
ctx.close()


In [ ]:
# We then remove those who died or had covid:
big_data = big_data_df[(big_data_df.DIED == 0) & (big_data_df.COVID == 0)]

# Small fix: Drop men with pregnancy-related claims (side note: apparently women can get things called prostate cancer):
big_data = big_data.drop(big_data[(big_data.GENDER == 2) & (big_data.UMC_PREGNANCY == 1)].index)

# Remove censoring-point ages (some providers make everyone above this age exactly this age, also 
# removing (likely delivery-related) spike at 0 and 1 in this data)
big_data = big_data[(big_data.AGE >= 0) & (big_data.AGE != 89) & (big_data.AGE != 80) & (big_data.AGE > 1)]

# Check that year of birth and age match roughly:
big_data = big_data[abs((2020-big_data.YOB) - big_data.AGE) <= 1]

# We then have our big sample to draw from
big_data.describe()

In [ ]:
# Optional plot of density for the two 
print(big_data.AGE.plot.density(color='blue'))
print(mdx.AGE.plot.density(color='red'))

In [ ]:
# Set up sample to add to the existing set of covid-positive:
# Generate age groups:
age_bucket_labels = ['0-4 Years', '5-14 Years', '15-24 Years', '25-34 Years', '35-44 Years',
                     '45-54 Years', '55-64 Years', '65-74 Years', '75-84 Years',
                     '85 Years and Over']
age_bucket_bins = [0, 5, 15, 25, 35, 45, 55, 65, 75, 85, 150]

big_data['AGE_GROUP'] = pd.cut(big_data.loc[:,'AGE'], age_bucket_bins, labels=age_bucket_labels)

big_data['WEEKS_SINCE_2020'] = 0

big_data.describe()   

In [ ]:
for age_group in age_bucket_labels:
    print(age_group + ": " + str(round(sum(cfr_merged.N_SAMPLES_NEEDED[(cfr_merged.N_SAMPLES_NEEDED > 0) 
                                                                 & (cfr_merged.AGE_GROUP == age_group)]))) + 
          " samples needed -- as %: " + 
          str(100*sum(cfr_merged.N_SAMPLES_NEEDED[(cfr_merged.N_SAMPLES_NEEDED > 0) 
                                              & (cfr_merged.AGE_GROUP == age_group)])/
              sum(cfr_merged.CASES_mdx[(cfr_merged.AGE_GROUP == age_group)])) + 
          " --- Obs in data: " + str(sum(cfr_merged.CASES_mdx[(cfr_merged.AGE_GROUP == age_group)])))


In [ ]:

# Set baseline to not plausible covid
big_data['PLAUSIBLE_COVID'] = 0
big_data['WEEKS_SINCE_2020'] = 0

# Make target df
extra_obs = []

# Randomly flip people into plausible covid:
added_samples = 0
ind = 0

for week in cfr_merged.WEEKS_SINCE_2020.unique():
    for age_group in age_bucket_labels:
        for gender in [1,2]:
            # Get number of plausible covid-19 patients in this intersection
            ntarget = cfr_merged.loc[(cfr_merged.AGE_GROUP == age_group) &
                                     (cfr_merged.GENDER == gender) &
                                     (cfr_merged.WEEKS_SINCE_2020 == week),'N_SAMPLES_NEEDED']
            
                
            # If ntarget is an integer, then proceed to add:
            try:
                ntarget = int(round(ntarget))
                
                print("Ntarget = " + str(ntarget) + " added samples total = " + str(added_samples) + 
                      " ***** progress =" + str(round(ind/len(cfr_merged.index)*10000)/100) +
                      " ***** age group = " + age_group + " and week = " + str(week))
                
                if ntarget > 0:

                    added_samples = added_samples + ntarget
                    
                    temp = big_data[(big_data.PLAUSIBLE_COVID == 0) &
                        (big_data.AGE_GROUP == age_group) & 
                        (big_data.GENDER == gender)].sample(ntarget, replace = True)
                    temp['WEEKS_SINCE_2020'] = week
                    temp['PLAUSIBLE_COVID'] = 1
                    extra_obs.append(temp)
            except:
                ""

            ind = ind + 1
           
# Concatenate all dataframes created above:
extra_obs = pd.concat(extra_obs).reset_index(drop=True)
extra_obs.describe()

In [ ]:
# Combine into the big dataset:
extra_obs['DAYS_SINCE_2020'] = extra_obs['WEEKS_SINCE_2020']*7

mdx_augmented = mdx.append(extra_obs)
mdx_augmented.describe()  

In [ ]:
# Check what age curve looks like for raw vs augmented:

CFR_by_age_raw = []
for i in range(0, 89):
    #print(np.mean(mdx_augmented.DIED[(mdx_augmented.AGE == i)]))
    CFR_by_age_raw.append(np.mean(mdx.DIED[(mdx.AGE == i)]))
    

CFR_by_age = []
for i in range(0, 89):
    #print(np.mean(mdx_augmented.DIED[(mdx_augmented.AGE == i)]))
    CFR_by_age.append(np.mean(mdx_augmented.DIED[(mdx_augmented.AGE == i)]))
    
plt.plot(CFR_by_age_raw, color = 'gray')
plt.plot(CFR_by_age, color = 'blue')
plt.title("Debiased data (blue) vs raw data (gray)")
plt.show()


plt.plot(CFR_by_age_raw, color = 'gray')
plt.plot(CFR_by_age, color = 'blue')
plt.title("Debiased data (blue) vs raw data (gray) - log scale")
plt.yscale('log')
plt.show()
# Looks reasonable.

In [ ]:
# Look at distribution of added obs:
print(extra_obs.AGE.plot.density(color='blue'))

In [ ]:
# Save data:
mdf.to_csv('DEBIASED_DATA.csv')

In [ ]:
cfr_merged['N_MISSING_DEATHS'] = (cfr_merged['CASES_mdx'] * 
                                  (cfr_merged['CFR_ROLLING_mdx']-cfr_merged['CFR_ROLLING_usa']))/cfr_merged['CFR_ROLLING_usa']

# We save this, and used it to augment the hospitalizations data in a similar way.
cfr_merged.to_csv("DEBIASING_TARGETS.csv")
cfr_merged.describe()